<a href="https://colab.research.google.com/github/pizzab0y/skills-release-based-workflow/blob/main/recsys_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Q:**
Можно ли, и если можно, то как построить простую рекомендательную систему на основе перцептрона?

## **A:**
Простейший перцептрон это сенсоры (входные данные) --> полносвязный слой нейронов (dense - то слово, которое вылетело у меня из головы во время созвона, - слой) --> выходной слой. Можно ли построить на его основе RecSys – да, можно. Это легко. На входе у нас будут какие-либо данные о пользователях. Которые по нашим предположениям так или иначе должны влиять на то, что им нравится. А на выходе мы получим некалиброванную вероятность класса чего-то, что пользователю должно “зайти”.

Итак, задача. Специально для тебя, Эдуард. У нас есть небольшая крафтовая пивоварня, а при ней - небольшой бар. Пивоварня в течении нескольких лет варит ровно 10 сортов пива, приведенных ниже. Бармен настолько устал слышать фразу "Чо порекомендуешь?", что решил создать небольшой скрипт на Python, который бы делал это за него.

Бармен предположил, что вкусовые предпочтения зависят от гостя. И придумал дизайн нехитрого исследования. Вот его суть: каждый новый посетитель за умеренную плату может поучаствовать в дегустации всех 10 сортов пива. С двумя условиями - на входе он заполняет простую анкету о себе по вопросам бармена. А на выходе, попробовав все сорта пива, выбирает одно лучшее по его мнению. Бармен решил использовать т.н. "жадный алгоритм" и не усложнять. Сэмплы пива всегда подаются в рандомном порядке.

За некоторое время работы бармен собирал заполенные анкеты и понравившийся сорт в папочку, которую с гордостью называл "Биг Дата".

Разумеется, все это - большая абстракция. Исследование далеко от научного. Тем не менее, это простейшая рекомендательная система, основывающейся на предположении, что похожим людям будет нравится похожее пиво (user-based collaborative system). Baseline моделью бармен выбрал модель KNN, что является довольно традиционным подходом в этом деле. Но, будучи сам натурой тонкой, бармен интуитивно понимал, что механизмы, по которым одним людям нравится одно пиво, а другим - другое, могут быть куда сложнее, поэтому и использовал Deep Learning. Вообще, в подобных задачах хорошо себя показывают бустинговые модели, но перцептрон, так перцептрон. Речь, напомню, идет о возможности создания, а не о качестве.

![Beer Tasting](https://www.amsterdambachelor.com/wp-content/uploads/2017/09/Amsterdam-Beer-Tasting.png)

In [1]:
beer = ["Lager", "IPA", "Stout", "Ale", "Pilsner", "Wheat Beer", "Pale Ale", "Saison", "Amber Ale", "Sours"]

survey = ["love_meat",
          "love_fish",
          "hard_work",
          "introvert",
          "love_football",
          "love_basketball",
          "do_sports",
          "drunk"]

Поехали!

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
RS = 404

Создам данные в папке бармена. К сожалению, они будут синтетические и распределены равномерно. Если наделять их какой-то логикой, то это займет много времени и уже не в рамках этого вопроса.

In [4]:
prng = np.random.RandomState(RS)

df = pd.DataFrame(data=prng.randint(2, size=(1000, 8)), columns=survey)
df['fav_beer'] = prng.choice(beer, 1000)

df.head(10)

,love_meat,love_fish,hard_work,introvert,love_football,love_basketball,do_sports,drunk,fav_beer
0,1,0,0,0,0,1,1,1,Stout
1,1,1,1,0,1,0,1,1,Sours
2,1,1,1,0,1,0,1,1,IPA
3,1,1,1,1,0,1,0,0,Pale Ale
4,0,0,0,1,1,0,0,0,Sours
5,1,0,0,0,1,1,0,0,Amber Ale
6,0,0,1,0,1,1,0,1,Pilsner
7,1,1,1,0,0,1,0,1,Stout
8,0,1,0,0,0,0,1,1,Sours
9,1,1,1,1,0,0,0,0,Pilsner


Отделим, разделим.

In [5]:
X = df.drop('fav_beer', axis=1)
y = df['fav_beer']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=1/5,
                                                    shuffle=True,
                                                    stratify=y,
                                                    random_state=RS)

print(* [_.shape for _ in [X_train, X_test, y_train, y_test]], sep='\n')

(800, 8)
(200, 8)
(800,)
(200,)


Закодируем labels.

In [6]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test);

Многослойный перцептрон в студию:

In [61]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu, input_dim=8),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [62]:
model.fit(X_train, np.eye(10)[y_train], epochs=10, batch_size=32)

Epoch 1/20
25/25 [==============================] - 1s 3ms/step - loss: 2.3234 - accuracy: 0.0800
Epoch 2/20
25/25 [==============================] - 0s 3ms/step - loss: 2.3026 - accuracy: 0.0975
Epoch 3/20
25/25 [==============================] - 0s 3ms/step - loss: 2.2934 - accuracy: 0.1175
Epoch 4/20
25/25 [==============================] - 0s 3ms/step - loss: 2.2859 - accuracy: 0.1238
Epoch 5/20
25/25 [==============================] - 0s 3ms/step - loss: 2.2779 - accuracy: 0.1450
Epoch 6/20
25/25 [==============================] - 0s 3ms/step - loss: 2.2711 - accuracy: 0.1612
Epoch 7/20
25/25 [==============================] - 0s 3ms/step - loss: 2.2663 - accuracy: 0.1675
Epoch 8/20
25/25 [==============================] - 0s 3ms/step - loss: 2.2600 - accuracy: 0.1800
Epoch 9/20
25/25 [==============================] - 0s 4ms/step - loss: 2.2550 - accuracy: 0.1787
Epoch 10/20
25/25 [==============================] - 0s 4ms/step - loss: 2.2504 - accuracy: 0.1813
Epoch 11/20
25/25 [

Модель обучилась и готова вовсю предсказывать.

In [68]:
pred_prob = model.predict(X_test)
y_pred = np.argmax(pred_prob, axis=1)
y_pred_beer = le.inverse_transform(y_pred)

print("Predicted classes:", y_pred_beer)

7/7 [==============================] - 0s 3ms/step
Predicted classes: [0 8 0 0 8 0 4 1 8 8 4 8 6 8 8 1 0 4 4 6 6 6 0 4 6 4 0 6 8 4 1 7 0 8 8 8 6
 3 8 1 8 9 0 4 1 8 1 6 0 6 4 1 8 6 8 0 8 9 1 9 1 8 8 5 8 9 0 8 4 9 8 0 1 6
 0 4 0 4 6 6 4 4 5 0 6 8 8 9 0 4 8 8 2 8 9 4 4 8 9 8 1 4 1 4 6 8 4 4 9 6 9
 2 0 8 4 0 9 8 8 0 0 8 8 1 8 8 8 8 1 1 4 1 0 5 9 1 4 8 1 4 1 3 9 4 8 9 0 0
 8 9 8 0 0 6 9 0 2 4 5 4 1 6 0 4 4 6 6 8 9 8 8 8 0 6 4 2 0 9 0 8 4 8 8 6 6
 0 8 2 9 4 8 0 6 9 8 4 1 0 7 8]


In [69]:
loss, accuracy = model.evaluate(X_test, np.eye(10)[y_test])

print("Test accuracy:", accuracy)

7/7 [==============================] - 0s 3ms/step - loss: 2.3339 - accuracy: 0.1000
Test accuracy: 0.10000000149011612


Чуда, как и ожидалось, не произошло( Но зато мы увидели как мог бы работать многослойный перцептрон в качестве простой рекомендательной системы.